In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3096296 sha256=39d26026705862c2334aa6a2b5504250af790d6ebd2740f285e63ab0bd588e1a
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import surprise

print(surprise.__version__)

1.1.3


In [ ]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [ ]:
data = Dataset.load_builtin('ml-100k')
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [ ]:
algo = SVD(random_state=0)
algo.fit(trainset)

In [ ]:
# 테스트 데이터 세트 전체에 대해 추천 영화 평점 데이터를 생성
# uid = userid / iid = itemid(영화 또는 아이템) / r_ui - 실제 평점
# details - 내부 처리 시 추천 예측을 할 수 없는 경우에 로그용으로 데이터를 남기는 데 사용됨
# test()는 사용자-아이템 평점 데이터 세트 전체에 대해서 추천을 예측하는 메서드임

predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

In [ ]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [ ]:
# predict()는 개별 사용자의 아이템에 대한 추천 평점을 예측해 줌
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함.
# 이때 사용자, 아이템 아이디는 문자열로 입력해야 함

uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [ ]:
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

#### Surprise 주요 모듈 소개

In [ ]:
import pandas as pd

# ratings = pd.read_csv('../dataset/ml-latest-small/ratings.csv')
ratings = pd.read_csv('/content/drive/Othercomputers/내 컴퓨터/hmkd1/dataset/ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.
# ratings.to_csv('../dataset/ml-latest-small/ratings_noh.csv', index=False, header=False)
ratings.to_csv('/content/drive/Othercomputers/내 컴퓨터/hmkd1/dataset/ml-latest-small/ratings_noh.csv', index=False, header=False)

In [ ]:
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


NameError: ignored

In [ ]:
# reader - 파일 포맷을 지정하는 매개변수
# 컬럼 순서가 사용자 ID, 아이템 ID, 평점 값 순으로 구성되어 있어야 함
# 이것을 line_format으로 지정해 줌
# sep - 데이터 파일의 열을 구분하는 구분자를 지정함, 여기서는 ,임
# rating_scale - 데이터셋에서 사용되는 평점 척도임, 0.5~5까지 (default는 1~5임)
# Dataset.load_from_df(df, reader)

from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file('../dataset/ml-latest-small/ratings_noh.csv',reader=reader)

In [ ]:
for rating in data.raw_ratings[:3]:
    print(rating)

('1', '1', 4.0, '964982703')
('1', '3', 4.0, '964981247')
('1', '6', 4.0, '964982224')


In [ ]:
# SVD 행렬 분해 기법을 이용한 추천 예측
# 잠재 요인 크기 K 값을 나타내는 파라미터인 n_factors -> 50으로 설정

trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [ ]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/Othercomputers/내 컴퓨터/hmkd1/dataset/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [ ]:
ratings.head(2)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247


### 베이스라인 평점 - 추후 공부

### 교차 검증과 하이퍼 파라미터 튜닝
####SVD 클래스의 입력 파라미터 (앞의 두 파라미터의 투닝 효과는 미미함, biased는 True 설정 유지가 좋음)
- n_factors - 잠재 요인 K의 개수. 디폴트는 100. 커질수록 정확도가 높아질 수 있으나 과적합 문제가 발생할 수 있음
- n_epochs - SGD(Stochastic Gradient Descent) 수행 시 반복 횟수. 디폴트는 20.
- biased (bool) - 베이스라인 사용자 편향 적용 여부이며, 디폴트는 True

In [ ]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('../dataset/ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8795  0.8767  0.8799  0.8680  0.8691  0.8746  0.0051  
MAE (testset)     0.6755  0.6698  0.6766  0.6641  0.6705  0.6713  0.0045  
Fit time          1.01    1.04    1.01    1.02    1.00    1.02    0.01    
Test time         0.14    0.15    0.14    0.15    0.15    0.14    0.00    


{'test_rmse': array([0.87950994, 0.87674993, 0.87986282, 0.86802023, 0.8691037 ]),
 'test_mae': array([0.67553974, 0.66976448, 0.67655478, 0.66411041, 0.67048417]),
 'fit_time': (1.0114214420318604,
  1.043590784072876,
  1.0074329376220703,
  1.0174188613891602,
  1.0024268627166748),
 'test_time': (0.13793063163757324,
  0.14991497993469238,
  0.14091157913208008,
  0.14891552925109863,
  0.14691662788391113)}

In [ ]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8757736151576433
{'n_epochs': 20, 'n_factors': 50}


#### Surprise 를 이용한 개인화 영화 추천 시스템 구축

In [ ]:
# 다음 코드는 train_test_split( )으로 분리되지 않는 데이터 세트에 fit( )을 호출해 오류가 발생합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

In [ ]:
# DatasetAutoFolds - 데이터 세트 전체를 학습 데이터로 사용하게 해 줌
# build_full_trainset() - 위의 클래스로 객체를 생성한 뒤, 이 메서드를 호출하면 전체 데이터를
# 학습 데이터 세트로 만들 수 있음

from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성.
data_folds = DatasetAutoFolds(ratings_file='../dataset/ml-latest-small/ratings_noh.csv', reader=reader)

#전체 데이터를 학습데이터로 생성함.
trainset = data_folds.build_full_trainset()

In [ ]:
ratings_noh

NameError: ignored

In [ ]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [ ]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('../dataset/ml-latest-small/movies.csv')

# userId=9 의 movieId 데이터 추출하여 movieId=42 데이터가 있는지 확인.
movieIds = ratings[ratings['userId']==9]['movieId']
if movieIds[movieIds==42].count() == 0:
    print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [ ]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [ ]:
# Surprise 내부의 데이터 객체에 대한 액세스 제약 등으로 인해 앞 절에서 사용한 get_unseen_movies()는
# 사용하지 않고 새롭게 get_unseen_surprise() 함수를 만들어 사용할 것임

def get_unseen_surprise(ratings, movies, userId):
    #입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
    seen_movies = ratings[ratings['userId']== userId]['movieId'].tolist()

    # 모든 영화들의 movieId를 리스트로 생성.
    total_movies = movies['movieId'].tolist()

    # 모든 영화들의 movieId중 이미 평점을 매긴 영화의 movieId를 제외하여 리스트로 생성
    unseen_movies= [movie for movie in total_movies if movie not in seen_movies]
    print('평점 매긴 영화수:',len(seen_movies), '추천대상 영화수:',len(unseen_movies), \
          '전체 영화수:',len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742


In [ ]:
# 추천 대상 영화 list
# [Prediction(uid='9', iid='1', r_ui=None, est=3.6398015009715396, details={'was_impossible': False}),
#  Prediction(uid='9', iid='2', r_ui=None, est=3.0708535569743463, details={'was_impossible': False}),...]

def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):
    # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
    predictions = [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]
    # predictions list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
    # [Prediction(uid='9', iid='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]
    # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
    # sortkey_est 함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
    def sortkey_est(pred):
        return pred.est

    # sortkey_est( ) 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
    predictions.sort(key=sortkey_est, reverse=True)    # 내림차순임
    top_predictions= predictions[:top_n]

    # top_n으로 추출된 영화의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
    top_movie_ids = [ int(pred.iid) for pred in top_predictions]
    top_movie_rating = [ pred.est for pred in top_predictions]
    top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']
    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)
print('##### Top-10 추천 영화 리스트 #####')

for top_movie in top_movie_preds:
    print(top_movie[1], ":", top_movie[2])

평점 매긴 영화수: 46 추천대상 영화수: 9696 전체 영화수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957


In [ ]:
top_movie_preds

[(858, 'Usual Suspects, The (1995)', 4.306302135700814),
 (260, 'Star Wars: Episode IV - A New Hope (1977)', 4.281663842987387),
 (296, 'Pulp Fiction (1994)', 4.278152632122759),
 (1196, 'Silence of the Lambs, The (1991)', 4.226073566460876),
 (50, 'Godfather, The (1972)', 4.1918097904381995),
 (1104, 'Streetcar Named Desire, A (1951)', 4.154746591122658),
 (1210,
  'Star Wars: Episode V - The Empire Strikes Back (1980)',
  4.122016128534504),
 (1213,
  'Star Wars: Episode VI - Return of the Jedi (1983)',
  4.108009609093436),
 (1242, 'Goodfellas (1990)', 4.083464936588478),
 (593, 'Glory (1989)', 4.07887165526957)]